In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
# from pyspark.ml.evaluation import RegressionEvaluator
import matplotlib.pyplot as plt
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import f1_score, recall_score, classification_report
from pyspark.sql.functions import col

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("LogisticRegression").getOrCreate()
    return spark
spark_object = init_spark()

In [3]:
train_spark = spark_object.read.option("delimiter", ",").csv("./data/trainbalanced.csv", header='true', inferSchema='true')

In [4]:
#test_spark = spark_object.read.option("delimiter", ",").csv("./data/testbalanced.csv", header='true', inferSchema='true')

In [5]:
# for i in train_spark.columns:
#     print(i)
# #     oldname = i
# #     if "." in oldname:
# #         newname = oldname.replace(".","")
# #         train_spark = train_spark.withColumnRenamed(oldname, newname)

TransactionID
TransactionDT
TransactionAmt
card1
card2
card3
card5
C1
C2
C3
C4
C5
C6
C7
C8
C9
C10
C11
C12
C13
C14
D1
V95
V96
V97
V98
V99
V100
V101
V102
V103
V104
V105
V106
V107
V108
V109
V110
V111
V112
V113
V114
V115
V116
V117
V118
V119
V120
V121
V122
V123
V124
V125
V126
V127
V128
V129
V130
V131
V132
V133
V134
V135
V136
V137
V167
V168
V169
V170
V171
V172
V173
V174
V175
V176
V177
V178
V179
V180
V181
V182
V183
V184
V185
V186
V187
V188
V189
V190
V191
V192
V193
V194
V195
V196
V197
V198
V199
V200
V201
V202
V203
V204
V205
V206
V207
V208
V209
V210
V211
V212
V213
V214
V215
V216
V220
V221
V222
V227
V234
V238
V239
V245
V250
V251
V255
V256
V259
V270
V271
V272
V279
V280
V281
V282
V283
V284
V285
V286
V287
V288
V289
V290
V291
V292
V293
V294
V295
V296
V297
V298
V299
V300
V301
V302
V303
V304
V305
V306
V307
V308
V309
V310
V311
V312
V313
V314
V315
V316
V317
V318
V319
V320
V321
id_01
id_02
id_11
id_17
id_19
id_20
ProductCD_C
ProductCD_H
ProductCD_R
ProductCD_S
card4_american express
card4_discover
card4_

In [6]:
#train_data, test_data = train_spark, test_spark

In [4]:
train_data, test_data = train_spark.randomSplit([0.7, 0.3], seed=123)
#train_data.groupBy(col("ProductCD_NUMERIC_imputed")).count().show()

In [5]:
input_list =[]
for i in train_data.columns:
    if i != 'isFraud':
        input_list.append(i)

In [6]:
assembler = VectorAssembler(inputCols=input_list,outputCol='features')

In [7]:
train_features = assembler.transform(train_data)
test_features = assembler.transform(test_data)

In [9]:
lr_models=[]
max_iters=[100, 200, 500]

#for nr in num_rounds:
for mi in max_iters:
    print("-"*25)
    print("max_iters: " + str(mi))
    lr = LogisticRegression(featuresCol='features', labelCol='isFraud', maxIter=mi, regParam=0.3, elasticNetParam=0.8)
    model = lr.fit(train_features)
    predictions = model.transform(train_features)
    y_train_pred=predictions.select("prediction").collect()
    y_train_orig=predictions.select("isFraud").collect()
    print('Training Report')
    print(classification_report(y_train_orig, y_train_pred))

    predictions = model.transform(test_features)
    y_test_pred=predictions.select("prediction").collect()
    y_test_orig=predictions.select("isFraud").collect()
    print('Test Report')
    print(classification_report(y_test_orig, y_test_pred))
    lr_models.append(model)

-------------------------
max_iters: 100
Training Report


C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     93211
           1       0.50      1.00      0.67     93300

    accuracy                           0.50    186511
   macro avg       0.25      0.50      0.33    186511
weighted avg       0.25      0.50      0.33    186511

Test Report


C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     39704
           1       0.50      1.00      0.67     39615

    accuracy                           0.50     79319
   macro avg       0.25      0.50      0.33     79319
weighted avg       0.25      0.50      0.33     79319

-------------------------
max_iters: 200
Training Report


...

C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     39704
           1       0.50      1.00      0.67     39615

    accuracy                           0.50     79319
   macro avg       0.25      0.50      0.33     79319
weighted avg       0.25      0.50      0.33     79319

-------------------------
max_iters: 500
Training Report


C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     93211
           1       0.50      1.00      0.67     93300

    accuracy                           0.50    186511
   macro avg       0.25      0.50      0.33    186511
weighted avg       0.25      0.50      0.33    186511

Test Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00     39704
           1       0.50      1.00      0.67     39615

    accuracy                           0.50     79319
   macro avg       0.25      0.50      0.33     79319
weighted avg       0.25      0.50      0.33     79319



C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# lr = LogisticRegression(featuresCol='features', labelCol='isFraud', maxIter=10, regParam=0.3, elasticNetParam=0.8)
# lr_model = lr.fit(train_features)

In [32]:
# #Training Metrics
# lr_train_predictions = lr_model.transform(train_features)
# y_pred=lr_train_predictions.select("prediction").collect()
# y_orig=lr_train_predictions.select("isFraud").collect()
# print(classification_report(y_orig, y_pred));

C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00     93211
           1       0.50      1.00      0.67     93300

    accuracy                           0.50    186511
   macro avg       0.25      0.50      0.33    186511
weighted avg       0.25      0.50      0.33    186511



C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [33]:

# lr_train_predictions.groupBy(col("prediction")).count().show()

+----------+------+
|prediction| count|
+----------+------+
|       1.0|186511|
+----------+------+



In [21]:
# lr_predictions = lr_model.transform(test_features)
# y_pred=lr_predictions.select("prediction").collect()
# y_orig=lr_predictions.select("isFraud").collect()
# print(classification_report(y_orig, y_pred));

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     27892
           1       0.50      1.00      0.67     27726

    accuracy                           0.50     55618
   macro avg       0.25      0.50      0.33     55618
weighted avg       0.25      0.50      0.33     55618



C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
# y_pred=lr_predictions.select("prediction").collect()
# y_orig=lr_predictions.select("isFraud").collect()

In [39]:
# print(classification_report(y_orig, y_pred));

              precision    recall  f1-score   support

           0       0.92      1.00      0.96     39475
           1       0.00      0.00      0.00      3375

    accuracy                           0.92     42850
   macro avg       0.46      0.50      0.48     42850
weighted avg       0.85      0.92      0.88     42850



C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amrit\anaconda3\envs\soen6111\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# accuracy = lr_predictions.filter(lr_predictions.isFraud == lr_predictions.prediction).count() / float(lr_predictions.count())
# print("Accuracy : ",accuracy)


Accuracy :  0.9212368728121354


In [16]:
# print(f1_score(y_orig, y_pred))
# print(recall_score(y_orig, y_pred))
# #print(accuracy_score(y_orig, y_pred))

0.0
0.0


NameError: name 'accuracy_score' is not defined

In [ ]:
# evaluator = RegressionEvaluator(labelCol='isFraud', predictionCol='prediction', metricName='f1')
# rmse = evaluator.evaluate(lr_predictions)
# print('Root Mean Squared Error (RMSE) =', rmse)

In [ ]:
# predictionAndLabels = lr_predictions.select("prediction","isFraud").rdd
# multi_metrics = MulticlassMetrics(predictionAndLabels)

In [ ]:
# f1_score = multi_metrics.fMeasure()

In [ ]:
# pr = trainingSummary.pr.toPandas()
# plt.plot(pr['recall'],pr['precision'])
# plt.ylabel('Precision')
# plt.xlabel('Recall')
# plt.show()